# Multi Agent Team Approach - With Termination Condition

We will be using 3 agents to now write the story, each with its own work and mastery

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os
from autogen_agentchat.agents import (AssistantAgent)

from autogen_agentchat.messages import TextMessage

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)
# model_client_2 = 

In [2]:
plot_agent = AssistantAgent(
    name = 'plot_writer',
    model_client=model_client,
    system_message="You create engaging plots for stories. Focus on the Pokemon's journey"
)

character_agent = AssistantAgent(
    name = 'character_writer',
    model_client=model_client,
    system_message="You develop characters. Describe the pokemon and the villian in detail, including their motivations and backgrounds."
)

ending_agent = AssistantAgent(
    name = 'ending_writer',
    model_client=model_client,
    system_message="You wrute engaging endings. conclude the story with a twist."
)

review_agent = AssistantAgent(
    name = 'review_writer',
    model_client=model_client,
    system_message="You have to say 'APPROVE' once the story is complete and you like the full flow else provide your feedback."
)


RoundRobinGroupChat is a simple yet effective team configuration where all agents share the same context and take turns responding in a round-robin fashion. Each agent, during its turn, broadcasts its response to all other agents, ensuring that the entire team maintains a consistent context.

In [3]:
from autogen_agentchat.conditions import MaxMessageTermination,TextMentionTermination

termination_condition = MaxMessageTermination(10) | TextMentionTermination('APPROVE')

In [4]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [plot_agent, character_agent, ending_agent,review_agent],
    termination_condition=termination_condition
    # max_turns=3
)
team


In [5]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='127c1b41-c346-47a8-8e1a-ea8a3da79402' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 2, 9, 46, 14, 961652, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'
id='90dc4fc4-2cb2-4cf8-ba9b-f8f393821b69' source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=111) metadata={} created_at=datetime.datetime(2025, 8, 2, 9, 46, 19, 827901, tzinfo=datetime.timezone.utc) content="Leaves of amber, gold, and red,  \nDance gracefully to where they're led.  \nIn autumn's whisper, breezes sing,  \nThe harvest moon, a silver ring.  \n\nCrisp air carries tales untold,  \nAs nature dons her cloak of gold.  \nPumpkins rest beneath the trees,  \nIn fall's embrace, the world finds ease.  \n\nUnderneath a sky so clear,  \nThe earth prepares for winter's near.  \nYet in each rustling leaf's descent,  \nAutumn's magic is truly spent.  " type='TextMessage'
id='0bdb0e39-5dfd-439d-a813-6d1e0b1ccb1a'

source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 6, 19, 20, 34, 54, 826785, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season.' type='TextMessage'


source='plot_writer' models_usage=RequestUsage(prompt_tokens=34, completion_tokens=130) metadata={} created_at=datetime.datetime(2025, 6, 19, 20, 34, 58, 973158, tzinfo=datetime.timezone.utc) content="Leaves of amber, gold, and rust,\nDance down gently, in the breeze’s trust.\nA whispering wind with a crisp embrace,\nSignals autumn’s gentle grace.\n\nPumpkin patches and apple pies,\nUnderneath the soft, gray skies.\nSweaters warm, and fires bright,\nGuide us through the cooler night.\n\nSquirrels gather with busy haste,\nPreparing for the chill they’ll face.\nA symphony of crunch beneath our feet,\nNature’s painting, bittersweet.\n\nThe world slows down, a tranquil scene,\nBeneath the autumn's vibrant sheen.\nIn this season, calm and wise,\nWe find a world in sweet disguise." type='TextMessage'


source='character_writer' models_usage=RequestUsage(prompt_tokens=178, completion_tokens=510) metadata={} created_at=datetime.datetime(2025, 6, 19, 20, 35, 11, 117737, tzinfo=datetime.timezone.utc) content='### Pokémon: Aurafrost\n\n**Description and Abilities:**  \nAurafrost is an Ice/Ghost type Pokémon with a shimmering, translucent body resembling a delicate, frosted snowflake. Its core glows with an ethereal light, casting colorful illusions within the icy crystals that make up its form. Aurafrost can glide silently through the air, often leaving trails of frost in its wake. Its eyes are deep blue and seem to hold ancient wisdom. Known for its move "Polar Mirage," Aurafrost creates illusions that disorient opponents, while its signature ability, "Chill Veil," reduces the accuracy of all incoming attacks.\n\n**Background and Motivation:**  \nLegends say Aurafrost was born during the longest winter, formed from the union of snow spirits and northern lights. It is considered a guardian of balance, ensuring the icy domains are respected and preserved. Aurafrost is a protector of forgotten and hidden realms within the vast tundra, engaging only when its territory or the tranquility of its environment is threatened. Its motivation stems from a desire to maintain harmony within its habitat and protect the serenity of the icy expanses from those who seek to exploit or corrupt them.\n\n### Villain: Lord Niveus\n\n**Description and Traits:**  \nLord Niveus is a charismatic yet formidable human clad in sleek, silver armor adorned with intricate ice motifs. His eyes are as cold as the glaciers he commands, and his skin is pale from years spent in the shadow of the ice realms. He is known for his cunning intellect and his mastery over ice-based Pokémon, commanding respect and fear in equal measure. Niveus carries with him a dark, sapphire-tipped staff capable of summoning blizzards.\n\n**Background and Motivation:**  \nOnce a celebrated researcher dedicated to uncovering the mysteries of the arctic Pokémon, Niveus was drawn into darkness following a personal tragedy involving the loss of his family in a severe snowstorm. He became consumed with an obsession to control the very forces that took everything from him. This path of vengeance led him to discover forbidden ice magic, which he now uses to unleash winter’s fury upon those who stand against him. His ultimate goal is to conquer all regions by enshrouding them in perpetual winter, believing that only through such dominance can he protect the world from the unpredictability and cruelty of nature. Yet, deep down, there remains a shard of the compassionate scientist he once was, occasionally leading to internal conflict, especially when faced with reminders of his past benevolence.' type='TextMessage'


source='ending_writer' models_usage=RequestUsage(prompt_tokens=688, completion_tokens=671) metadata={} created_at=datetime.datetime(2025, 6, 19, 20, 35, 30, 995519, tzinfo=datetime.timezone.utc) content='### The Clash of Aurafrost and Lord Niveus\n\nIn the heart of the frosty tundra, where the auroras painted the sky with vibrant hues, a confrontation brewed. Aurafrost, the guardian of icy realms, floated silently amidst the crystalline landscape. Its presence was a gentle reminder of the balance it sought to maintain and a warning to those who dared disrupt the peace.\n\nLord Niveus, determined to bend the ice and snow to his will, approached the ethereal Pokémon. His intentions were clear: to harness Aurafrost\'s power to further his dominion over perpetual winter. As they faced each other, the air grew still, and a tension mounted, like the calm before a storm.\n\nNiveus raised his sapphire-tipped staff, summoning a blizzard that raged with his inner turmoil. The snow whipped around them, but Aurafrost remained unflinching, casting its "Polar Mirage" to distort the reality surrounding them. The illusions disoriented Niveus, drawing out the agony of choice in his heart—weapon or surrender, conquest or harmony.\n\nAs their icy battle raged, a glimmer of light flickered within Niveus’s mind, a fleeting memory of his family’s laughter amidst the snow. The reminder pulled at the remnants of the kindness he once embodied. The staff in his hand began to tremble, and for the briefest of moments, he hesitated.\n\nSensing the change, Aurafrost cast one final illusion: the image of Niveus, surrounded by the family he lost, their faces serene and free from fear. The mirage, hauntingly beautiful, penetrated his armored heart, igniting the slumbering compassion he had buried beneath layers of vengeance.\n\nBut just as he softened, the staff—now independently craving the dark power it wielded—besked with wicked frost, sparking a rogue surge of ice magic. The ground beneath Niveus cracked open, revealing a hidden, ancient chamber—a vault of precious winter artifacts—unseen by anyone for centuries.\n\nAs Niveus struggled to comprehend what lay beneath, Aurafrost swiftly moved, encasing the rogue staff in its icy grip. The vault had awakened due to the magic\'s disturbance, and within it lay artifacts capable of bringing about not just eternal winter, but such primal forces that could reshape the world in unintended ways.\n\nWith a sigh that resonated like the whispers of the tundra winds, Aurafrost shattered the staff, ending its malicious grasp on Niveus. The guardian, with its Chill Veil, sealed the vault once more, leaving Niveus standing silent amidst the settling snow.\n\nUnderstanding the gravity of his actions and awakening from the dark haze that clouded him, Niveus, stripped of his ambitions, gazed at Aurafrost in humility and awe. Choosing a path of restoration rather than ruin, he pledged to work alongside the guardian, dedicating himself to healing and safeguarding the majestic, icy domains he once sought to control.\n\nOnly Aurafrost, gliding silently away, knew the secrets of the hidden vault. Yet, unbeknownst to both of them, a shard of the dark ice magic had lingered, embedding itself subtly into the tundra—awaiting a new storm to rekindle its fury.' type='TextMessage'

source='review_writer' models_usage=RequestUsage(prompt_tokens=1378, completion_tokens=3) metadata={} created_at=datetime.datetime(2025, 6, 19, 20, 35, 32, 332636, tzinfo=datetime.timezone.utc) content='APPROVE' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned

In [6]:
from autogen_agentchat.base import TaskResult

# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season APPROVE."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)

id='74cccfbd-efed-4245-b8c2-3f69124a0d90' source='user' models_usage=None metadata={} created_at=datetime.datetime(2025, 8, 2, 9, 46, 57, 147510, tzinfo=datetime.timezone.utc) content='Write a short poem about the fall season APPROVE.' type='TextMessage'
Stop Reason: Text 'APPROVE' mentioned
